
GTFS Wiki: http://wiki.corp.nextbus.com/nextbuswiki/GtfsChecks (need to be on NexBus VPN) 

GTFS Goolge Guideline: https://developers.google.com/transit/gtfs/reference/

# <span style="color:red">Import the needed packages</span>

In [1]:
import pandas as pd
import numpy as np
#import pixiedust as pix # will look into the capabilites of this library

### <span style="color:green">Add the location of your GTFS data</span>

In [14]:
# for windows
# filepath = r"\\cts-con-fld-001\users$\207329\My Documents\qa\vacaville\vacavillecitycoach-ca-us_20181015"

# for linux
filepath = r"/mnt/shared/qa/foothill"

# filepath2 = r"/mnt/shared/qa/seattle-sc/Seattle-sc-ALL_20181126_google_daily_transit"



'''Some functions to make life eaiser
Note if you are using Windows, you'll need to modify the "/" '''




# load file and show top 5 records
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    global df
    df = pd.read_csv(f)
    return df.info()

# load second file for comparision
def load2(fName): 
    f = filepath2 + "/" + "{}.txt".format(fName)
    global df2
    df2 = pd.read_csv(f)
    return df2.info()

import csv
# short save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '//' + "{}".format(fName), sep =',', index=False)#, quoting=csv.QUOTE_NONNUMERIC)
    return print("Saved {} to {}".format(fName,filepath))

# short find null
def null(fName):
    return df[(fName).isnull()]

# drop field (field name, file name)
def drop(fName,fileName):
    df.drop(fName, 1, inplace=True)
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Dropped {} field.\n\
Saved '{}_FIXED.txt' to file folder.".format(fName,fileName))

# drop dups in blocks.txt for CNB 
def dups():
    #df[df.block_id.duplicated()]
    df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
    return df1.info()
    
# sort sequence for stop_times.txt file
def sort():
    df.sort_values(['trip_id', 'stop_sequence'])
    return df.head(15)

# shape less than certain amount of points for shapes
def lessthan(x):
    df1 = df.groupby(['shape_id']).count()
    return df1[(df1.shape_pt_lat < x)]

# function to move copy stop_code to stop_id in the stops.txt file
def copyField(newField,origField,fileName):
    df[newField] = df[origField]
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Copied the '{}' field to the '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(origField,newField,fileName))




### work for CNB shape issues
# def shapeFix(indx1,indx2,shapeName):
#     x1 = int(indx1)
#     x2 = int(indx2)
#     global df
#     df = df.drop(df.index[x1:x2]) # drop double shape pnts 
#     df = df[(df.shape_id == '{}'.format(shapeName))] # create new df of just shape pnts
#     df['shape_pt_sequence'] = range(1,1 + len(df)) # re-sequence shape pnts
#     df_cut = df # give unique name to subset of shape pnts

#     # load orig data, drop all of courthou1, append 
#     load('shapes') # load orig df
#     df = df[df.shape_id != '{}'.format(shapeName)] # drop all cases of shape 
#     global df_all
#     df_all = df.append(df_cut,ignore_index=True) # create new df and append df_cut
#     return  print('use df_all')



## Random Work-flows

In [ ]:
## fill na with number, query and output file  
df['block_id'].fillna(999999, inplace=True)
df1 = df[(df.block_id == 999999)]
df['block_id'].unique()
save(df1,'trips_missingBlocks.txt')

# compare two files
# set the field to a list
oldTrips = df2['trip_id'].unique().tolist()
newTrips = df['trip_id'].unique().tolist()

oldBlocks = df2['block_id'].unique().tolist()
newBlocks = df['block_id'].unique().tolist()

# use set and difference to compare the two list
set(oldTrips).difference(newTrips)

set(newBlocks).difference(oldBlocks)

# can alsow use this logic... but the above works just fine
if (40803723 in newTrips):
    print ("Exist")
else:
    print("Nope")

##### seattle-sc 'like' function 
df=df.loc[df['route_id'].isin([100340,102638])]
df
save(df,'trips')
df['service_id'].unique()
df2=df2.loc[df2['route_id'].isin([100340,102638])]

#### create new column based on conditions of other columns
df['timeCheck'] = np.where(df['arrival_time']==df['departure_time'],'yes','no')
df1 = df[(df.timeCheck=='no')]
df['timeCheck'].unique()

# syntax for leading zeros (4 for example)
df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))


# <span style="color:red">agency.txt</span>

In [28]:
load('agency')
# df.head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
agency_name        1 non-null object
agency_url         1 non-null object
agency_timezone    1 non-null object
agency_lang        0 non-null float64
agency_phone       1 non-null object
dtypes: float64(1), object(4)
memory usage: 120.0+ bytes


# <span style="color:red">routes.txt</span>

### Required Fields: 
1. route_id- These are the routes that are shown to the public. Route ID's cannot have leading zeros, asterisks, underscore, dashes and perhaps other characters. 
2. route_type- numberic value with specific meanings:
    * 0 Tram, Streetcar, Light rail
    * 1 Subway, Metro
    * 2 Rail
    * 3 Bus
    * 4 Ferry
    * 5 Cable Car

In [27]:
# just use the funtion 'load()' to load the .txt file and see the first 5 lines ex: loadfile('routes')
load('routes')
# df.head(100)
#copyField('route_short_name','route_id','routes')
# dfEOS = df[(df.agency_id == 'EOS')]
# save(dfEOS,'routes')

#### seattle-sc 'like' function
#df = df.loc[df['agency_id']=='EOS']
# df1['route_long_name'] = df1['route_short_name']
# df = df1
# save('routes_NEW.txt')
#df[(df.route_id==100249)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
route_id            39 non-null object
route_short_name    39 non-null object
route_long_name     39 non-null object
route_desc          0 non-null float64
route_type          39 non-null int64
route_url           39 non-null object
route_color         0 non-null float64
route_text_color    0 non-null float64
dtypes: float64(3), int64(1), object(4)
memory usage: 2.5+ KB


# <span style="color:red">trips.txt</span>

### Required Fields:
1. route_id- Check to see if there are trips for every route. 
2. service_id- Check to see if the value of this field comes from the calendar.txt file. (below)
3. trip_id
4. direction_id- Can either be 1 or 0. Can omit if all routes are in one direction. 
5. block_id- GTFS list this as optional, NextBus requires it. Make sure there are no missig values. 
6. shape_id- GTFS list this as optional, NextBus requires it. This field links with the shapes.txt file.(below)
   


In [29]:
load('trips')
# df.head()


# dffoo = df[(df.block_id == 5254117)]
# dffoo.head()

## dups work
#df[df.block_id.duplicated()]
#df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
#102638

# df1 = df[(df.route_id == '860-150') | (df.route_id == '861-150')]
# df1 = df[(df.trip_headsign == 'Not In Service')]
# df1
# save(df1,'foothill_trips_notInService.txt')




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4579 entries, 0 to 4578
Data columns (total 7 columns):
route_id         4579 non-null object
service_id       4579 non-null object
trip_id          4579 non-null object
trip_headsign    4579 non-null object
direction_id     4579 non-null int64
block_id         4579 non-null int64
shape_id         4579 non-null int64
dtypes: int64(3), object(4)
memory usage: 250.5+ KB


# <span style="color:red">stops.txt</span>

### Required Fields:
1. stop_id 
2. stop_lat- Make sure none are zero, or empty.
3. stop_lon- Make sure none are zero, or empty.
    - make sure lat and long are at least five decimal places (most do six) 

In [30]:
load('stops')
# df.head(25)

# copyField('stop_code','stop_id','stops')

#### seattle-sc 'like' function
#df_stop = df.loc[df['stop_code'].isin(stopList)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 10 columns):
stop_id           1971 non-null object
stop_code         0 non-null float64
stop_name         1971 non-null object
stop_desc         0 non-null float64
stop_lat          1971 non-null float64
stop_lon          1971 non-null float64
zone_id           0 non-null float64
stop_url          0 non-null float64
location_type     1971 non-null int64
parent_station    8 non-null object
dtypes: float64(6), int64(1), object(3)
memory usage: 154.1+ KB


# <span style="color:red">stop_times.txt</span>

### Required Fields:
1. trip_id
2. arrival_time- Should be blank for non-scheduled stops
3. depature_times- Should be blank for non-scheduled stops
4. stop_id
5. stop_sequence- Some agencies will sort this field incorrectly, putting all the "1s" first.
    - note* watch out for the timepoints field, if there is a 1, then it is an acutal stop. If there are not enough of 1s, this may cause problems. It's okay to delete this field. 

In [31]:
load('stop_times') 
# df.head(25)

#drop('timepoint','stop_times')
# df.head(100)
# syntax for leading zeros (4 for example)
# df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))
# df.head(100)
# df['timepoint'].unique()

# df[(df.arrival_time!=null) & (df.timepoint==1)]
# df[(df.trip_id == 't_31175_b_718_tn_0')]

# drop('timepoint','stop_times')




#### seattle-sc 'like' function
# df_trip = df.loc[df['trip_id'].isin(tripsList)]
# df_trip.sort_values(['trip_id', 'stop_sequence'])
# stopList = df_trip['stop_id'].unique().tolist()
# save(df_trip,'stop_times_NEW.txt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188235 entries, 0 to 188234
Data columns (total 9 columns):
trip_id                188235 non-null object
arrival_time           188235 non-null object
departure_time         188235 non-null object
stop_id                188235 non-null int64
stop_sequence          188235 non-null int64
pickup_type            188235 non-null int64
drop_off_type          188235 non-null int64
shape_dist_traveled    188235 non-null float64
timepoint              188235 non-null int64
dtypes: float64(1), int64(5), object(3)
memory usage: 12.9+ MB


# <span style="color:red">shapes.txt</span>

### Required Fields:
1. shape_id
2. shape_pnt_lat- at least five decimal places
3. shape_pnt_lon- at least five deciaml places

In [32]:
load('shapes')
# df.head(50)

# lessthan(100)


# df['shape_pt_sequence'] = df['shape_pt_sequence'].astype(np.int64)
# save(df,'shapesINT.txt')
# df1 = df[(df.shape_id=='J_trip_inbound_pm')]
# save(df1,'J_trip_inbound_pm.csv')

# flip the point sequence 
# df2 = df[(df.shape_id == 'J_trip_inbound_am')]
# drop = df2[::-1].drop(columns=['shape_pt_sequence'])
# drop.loc[:,'shape_pt_sequence'] = range(1, 1 + len(drop))
# # drop

# df3 = df[df.shape_id != 'J_trip_inbound_am']
# df4 = df3.append(drop)
# new = df4[(df4.shape_id == 'J_trip_inbound_am')]
# df4
# save(df4,'shapes_NEW.txt')
# dfJ = df[(df['shape_id'].str.contains('J'))]
# dfJ['shape_id'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76108 entries, 0 to 76107
Data columns (total 5 columns):
shape_id               76108 non-null int64
shape_pt_lat           76108 non-null float64
shape_pt_lon           76108 non-null float64
shape_pt_sequence      76108 non-null int64
shape_dist_traveled    76108 non-null float64
dtypes: float64(3), int64(2)
memory usage: 2.9 MB


# <span style="color:red">calendar.txt</span>

### Required:
Run the calendar script if more than one service record is defined for the day of the week.
* If there is more than one service defined for the week, then run the calendar.py script.


In [33]:
load('calendar')
# df.head(35)


#### seattle-sc 'like' function
# df_cal = df.loc[df['service_id'].isin(serviceList)]
# df_cal

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
service_id    8 non-null object
monday        8 non-null int64
tuesday       8 non-null int64
wednesday     8 non-null int64
thursday      8 non-null int64
friday        8 non-null int64
saturday      8 non-null int64
sunday        8 non-null int64
start_date    8 non-null int64
end_date      8 non-null int64
dtypes: int64(9), object(1)
memory usage: 720.0+ bytes


# <span style="color:red">calendar_dates.txt</span>

### Required:
This is an optional file, but is used to mark special holidays. Make sure the dates make sense.

In [34]:
load('calendar_dates')
# df.head(50)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
service_id        12 non-null object
date              12 non-null int64
exception_type    12 non-null int64
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes
